# Libraries

In [1]:
import gradio as gr
import pandas as pd
import numpy as np
# import json
# import io 
import sys
# from PIL import Image
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS

from datetime import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext gradio

# Recommendation generation

In [3]:
embeddings = HuggingFaceEmbeddings()

In [4]:
new_db = FAISS.load_local("../../vector_db/phase2_symbols", embeddings)

In [5]:
def get_recommendations(query, top_k):
  relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name'), relevant_stocks[idx][1]) for idx in range(len(relevant_stocks))]
  for idx, data in enumerate(recommendations):
    if data[2]>=0:
      print('{}. {} : {} |score : {}'.format(idx+1, data[0], data[1], data[2]))


In [6]:
stock_pref = 'businesses in the domain of software and Inforamation technology'

get_recommendations(stock_pref, top_k = 10)

1. ECL : E - CHANNELLING PLC |score : 0.15225744611490732
2. SLTL : SRI LANKA TELECOM PLC |score : 0.1514007692720748
3. GEST : GESTETNER OF CEYLON PLC |score : 0.09990819427867814
4. HAYL : HAYLEYS PLC |score : 0.0751765921598796
5. DIAL : DIALOG AXIATA PLC |score : 0.07034462456549961
6. LPL : LAUGFS POWER PLC |score : 0.06791991637066785
7. DIMO : DIESEL & MOTOR ENGINEERING PLC |score : 0.06789513402374348
8. HBS : hSenid Business Solutions PLC |score : 0.06200249883973841
9. MELS : MELSTACORP PLC |score : 0.060525841855310736
10. MDL : MYLAND DEVELOPMENTS PLC |score : 0.05051326793566868


# Collaborative filtering

In [7]:
import torch
import torch.nn as nn
from tqdm.auto import trange

from cornac.models.recommender import Recommender
from cornac.utils.common import scale
from cornac.exception import ScoreException

from cornac.models.recommender import ANNMixin, MEASURE_DOT


# cornac 1.18 BiVAE

## bivae

In [8]:
import itertools as it

import numpy as np
import torch
import torch.nn as nn
from tqdm.auto import trange


EPS = 1e-10

ACT = {
    "sigmoid": nn.Sigmoid(),
    "tanh": nn.Tanh(),
    "elu": nn.ELU(),
    "relu": nn.ReLU(),
    "relu6": nn.ReLU6(),
}


class BiVAE(nn.Module):
    def __init__(
        self,
        k,
        user_encoder_structure,
        item_encoder_structure,
        act_fn,
        likelihood,
        cap_priors,
        feature_dim,
        batch_size,
    ):
        super(BiVAE, self).__init__()

        self.mu_theta = torch.zeros((item_encoder_structure[0], k))  # n_users*k
        self.mu_beta = torch.zeros((user_encoder_structure[0], k))  # n_items*k

        self.theta = torch.randn(item_encoder_structure[0], k) * 0.01
        self.beta = torch.randn(user_encoder_structure[0], k) * 0.01
        torch.nn.init.kaiming_uniform_(self.theta, a=np.sqrt(5))

        self.likelihood = likelihood
        self.act_fn = ACT.get(act_fn, None)
        if self.act_fn is None:
            raise ValueError("Supported act_fn: {}".format(ACT.keys()))

        self.cap_priors = cap_priors
        if self.cap_priors.get("user", False):
            self.user_prior_encoder = nn.Linear(feature_dim.get("user"), k)
        if self.cap_priors.get("item", False):
            self.item_prior_encoder = nn.Linear(feature_dim.get("item"), k)

        # User Encoder
        self.user_encoder = nn.Sequential()
        for i in range(len(user_encoder_structure) - 1):
            self.user_encoder.add_module(
                "fc{}".format(i),
                nn.Linear(user_encoder_structure[i], user_encoder_structure[i + 1]),
            )
            self.user_encoder.add_module("act{}".format(i), self.act_fn)
        self.user_mu = nn.Linear(user_encoder_structure[-1], k)  # mu
        self.user_std = nn.Linear(user_encoder_structure[-1], k)

        # Item Encoder
        self.item_encoder = nn.Sequential()
        for i in range(len(item_encoder_structure) - 1):
            self.item_encoder.add_module(
                "fc{}".format(i),
                nn.Linear(item_encoder_structure[i], item_encoder_structure[i + 1]),
            )
            self.item_encoder.add_module("act{}".format(i), self.act_fn)
        self.item_mu = nn.Linear(item_encoder_structure[-1], k)  # mu
        self.item_std = nn.Linear(item_encoder_structure[-1], k)

    def to(self, device):
        self.beta = self.beta.to(device=device)
        self.theta = self.theta.to(device=device)
        self.mu_beta = self.mu_beta.to(device=device)
        self.mu_theta = self.mu_theta.to(device=device)
        return super(BiVAE, self).to(device)

    def encode_user_prior(self, x):
        h = self.user_prior_encoder(x)
        return h

    def encode_item_prior(self, x):
        h = self.item_prior_encoder(x)
        return h

    def encode_user(self, x):
        h = self.user_encoder(x)
        return self.user_mu(h), torch.sigmoid(self.user_std(h))

    def encode_item(self, x):
        h = self.item_encoder(x)
        return self.item_mu(h), torch.sigmoid(self.item_std(h))

    def decode_user(self, theta, beta):
        h = theta.mm(beta.t())
        return torch.sigmoid(h)

    def decode_item(self, theta, beta):
        h = beta.mm(theta.t())
        return torch.sigmoid(h)

    def reparameterize(self, mu, std):
        eps = torch.randn_like(mu)
        return mu + eps * std

    def forward(self, x, user=True, beta=None, theta=None):

        if user:
            mu, std = self.encode_user(x)
            theta = self.reparameterize(mu, std)
            return theta, self.decode_user(theta, beta), mu, std
        else:
            mu, std = self.encode_item(x)
            beta = self.reparameterize(mu, std)
            return beta, self.decode_item(theta, beta), mu, std

    def loss(self, x, x_, mu, mu_prior, std, kl_beta):
        # Likelihood
        ll_choices = {
            "bern": x * torch.log(x_ + EPS) + (1 - x) * torch.log(1 - x_ + EPS),
            "gaus": -(x - x_) ** 2,
            "pois": x * torch.log(x_ + EPS) - x_,
        }

        ll = ll_choices.get(self.likelihood, None)
        if ll is None:
            raise ValueError("Supported likelihoods: {}".format(ll_choices.keys()))

        ll = torch.sum(ll, dim=1)

        # KL term
        kld = -0.5 * (1 + 2.0 * torch.log(std) - (mu - mu_prior).pow(2) - std.pow(2))
        kld = torch.sum(kld, dim=1)

        return torch.mean(kl_beta * kld - ll)


def learn(
    bivae,
    train_set,
    n_epochs,
    batch_size,
    learn_rate,
    beta_kl,
    verbose,
    device=torch.device("cpu"),
    dtype=torch.float32,
):
    user_params = it.chain(
        bivae.user_encoder.parameters(),
        bivae.user_mu.parameters(),
        bivae.user_std.parameters(),
    )

    item_params = it.chain(
        bivae.item_encoder.parameters(),
        bivae.item_mu.parameters(),
        bivae.item_std.parameters(),
    )

    if bivae.cap_priors.get("user", False):
        user_params = it.chain(user_params, bivae.user_prior_encoder.parameters())
        user_features = train_set.user_feature.features[: train_set.num_users]

    if bivae.cap_priors.get("item", False):
        item_params = it.chain(item_params, bivae.item_prior_encoder.parameters())
        item_features = train_set.item_feature.features[: train_set.num_items]

    u_optimizer = torch.optim.Adam(params=user_params, lr=learn_rate)
    i_optimizer = torch.optim.Adam(params=item_params, lr=learn_rate)

    x = train_set.matrix.copy()
    x.data = np.ones_like(x.data)  # Binarize data
    tx = x.transpose()

    progress_bar = trange(1, n_epochs + 1, disable=not verbose)
    for _ in progress_bar:
        # item side
        i_sum_loss = 0.0
        i_count = 0
        for i_ids in train_set.item_iter(batch_size, shuffle=False):
            i_batch = tx[i_ids, :]
            i_batch = i_batch.A
            i_batch = torch.tensor(i_batch, dtype=dtype, device=device)

            # Reconstructed batch
            beta, i_batch_, i_mu, i_std = bivae(i_batch, user=False, theta=bivae.theta)

            i_mu_prior = 0.0  # zero mean for standard normal prior if not CAP prior
            if bivae.cap_priors.get("item", False):
                i_batch_f = item_features[i_ids]
                i_batch_f = torch.tensor(i_batch_f, dtype=dtype, device=device)
                i_mu_prior = bivae.encode_item_prior(i_batch_f)

            i_loss = bivae.loss(i_batch, i_batch_, i_mu, i_mu_prior, i_std, beta_kl)
            i_optimizer.zero_grad()
            i_loss.backward()
            i_optimizer.step()

            i_sum_loss += i_loss.data.item()
            i_count += len(i_batch)

            beta, _, i_mu, _ = bivae(i_batch, user=False, theta=bivae.theta)

            bivae.beta.data[i_ids] = beta.data
            bivae.mu_beta.data[i_ids] = i_mu.data

        # user side
        u_sum_loss = 0.0
        u_count = 0
        for u_ids in train_set.user_iter(batch_size, shuffle=False):
            u_batch = x[u_ids, :]
            u_batch = u_batch.A
            u_batch = torch.tensor(u_batch, dtype=dtype, device=device)

            # Reconstructed batch
            theta, u_batch_, u_mu, u_std = bivae(u_batch, user=True, beta=bivae.beta)

            u_mu_prior = 0.0  # zero mean for standard normal prior if not CAP prior
            if bivae.cap_priors.get("user", False):
                u_batch_f = user_features[u_ids]
                u_batch_f = torch.tensor(u_batch_f, dtype=dtype, device=device)
                u_mu_prior = bivae.encode_user_prior(u_batch_f)

            u_loss = bivae.loss(u_batch, u_batch_, u_mu, u_mu_prior, u_std, beta_kl)
            u_optimizer.zero_grad()
            u_loss.backward()
            u_optimizer.step()

            u_sum_loss += u_loss.data.item()
            u_count += len(u_batch)

            theta, _, u_mu, _ = bivae(u_batch, user=True, beta=bivae.beta)
            bivae.theta.data[u_ids] = theta.data
            bivae.mu_theta.data[u_ids] = u_mu.data

            progress_bar.set_postfix(
                loss_i=(i_sum_loss / i_count), loss_u=(u_sum_loss / (u_count))
            )

    # infer mu_beta
    for i_ids in train_set.item_iter(batch_size, shuffle=False):
        i_batch = tx[i_ids, :]
        i_batch = i_batch.A
        i_batch = torch.tensor(i_batch, dtype=dtype, device=device)

        beta, _, i_mu, _ = bivae(i_batch, user=False, theta=bivae.theta)
        bivae.mu_beta.data[i_ids] = i_mu.data

    # infer mu_theta
    for u_ids in train_set.user_iter(batch_size, shuffle=False):
        u_batch = x[u_ids, :]
        u_batch = u_batch.A
        u_batch = torch.tensor(u_batch, dtype=dtype, device=device)

        theta, _, u_mu, _ = bivae(u_batch, user=True, beta=bivae.beta)
        bivae.mu_theta.data[u_ids] = u_mu.data

    return bivae

## BiVAECF recommender

In [9]:
class BiVAECF(Recommender, ANNMixin):
    """Bilateral Variational AutoEncoder for Collaborative Filtering.

    Parameters
    ----------
    k: int, optional, default: 10
        The dimension of the stochastic user ``theta'' and item ``beta'' factors.

    encoder_structure: list, default: [20]
        The number of neurons per layer of the user and item encoders for BiVAE.
        For example, encoder_structure = [20], the user (item) encoder structure will be [num_items, 20, k] ([num_users, 20, k]).

    act_fn: str, default: 'tanh'
        Name of the activation function used between hidden layers of the auto-encoder.
        Supported functions: ['sigmoid', 'tanh', 'elu', 'relu', 'relu6']

    likelihood: str, default: 'pois'
        The likelihood function used for modeling the observations.
        Supported choices:

        bern: Bernoulli likelihood
        gaus: Gaussian likelihood
        pois: Poisson likelihood

    n_epochs: int, optional, default: 100
        The number of epochs for SGD.

    batch_size: int, optional, default: 100
        The batch size.

    learning_rate: float, optional, default: 0.001
        The learning rate for Adam.

    beta_kl: float, optional, default: 1.0
        The weight of the KL terms as in beta-VAE.

    cap_priors: dict, optional, default: {"user":False, "item":False}
        When {"user":True, "item":True}, CAP priors are used (see BiVAE paper for details),\
        otherwise the standard Normal is used as a Prior over the user and item latent variables.

    name: string, optional, default: 'BiVAECF'
        The name of the recommender model.

    trainable: boolean, optional, default: True
        When False, the model is not trained and Cornac assumes that the model is already \
        pre-trained.

    verbose: boolean, optional, default: False
        When True, some running logs are displayed.

    seed: int, optional, default: None
        Random seed for parameters initialization.

    use_gpu: boolean, optional, default: True
        If True and your system supports CUDA then training is performed on GPUs.

    References
    ----------
    * Quoc-Tuan Truong, Aghiles Salah, Hady W. Lauw. " Bilateral Variational Autoencoder for Collaborative Filtering."
    ACM International Conference on Web Search and Data Mining (WSDM). 2021.
    """

    def __init__(
        self,
        name="BiVAECF",
        k=10,
        user_encoder_structure=[20],
        item_encoder_structure = [20],
        act_fn="tanh",
        likelihood="pois",
        n_epochs=100,
        batch_size=100,
        learning_rate=0.001,
        beta_kl=1.0,
        cap_priors={"user": False, "item": False},
        trainable=True,
        verbose=False,
        seed=None,
        use_gpu=True,
    ):
        Recommender.__init__(self, name=name, trainable=trainable, verbose=verbose)
        self.k = k
        self.user_encoder_structure = user_encoder_structure
        self.item_encoder_structure = item_encoder_structure
        self.act_fn = act_fn
        self.likelihood = likelihood
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate
        self.beta_kl = beta_kl
        self.cap_priors = cap_priors
        self.seed = seed
        self.use_gpu = use_gpu


    def fit(self, train_set, val_set=None):
        """Fit the model to observations.

        Parameters
        ----------
        train_set: :obj:`cornac.data.Dataset`, required
            User-Item preference data as well as additional modalities.

        val_set: :obj:`cornac.data.Dataset`, optional, default: None
            User-Item preference data for model selection purposes (e.g., early stopping).

        Returns
        -------
        self : object
        """
        Recommender.fit(self, train_set, val_set)

        import torch
        # from .bivae import BiVAE, learn
        self.device = (
            torch.device("cuda:0")
            if (self.use_gpu and torch.cuda.is_available())
            else torch.device("cpu")
        )

        if self.trainable:
            feature_dim = {"user": None, "item": None}
            if self.cap_priors.get("user", False):
                if train_set.user_feature is None:
                    raise ValueError(
                        "CAP priors for users is set to True but no user features are provided"
                    )
                else:
                    feature_dim["user"] = train_set.user_feature.feature_dim

            if self.cap_priors.get("item", False):
                if train_set.item_feature is None:
                    raise ValueError(
                        "CAP priors for items is set to True but no item features are provided"
                    )
                else:
                    feature_dim["item"] = train_set.item_feature.feature_dim

            if self.seed is not None:
                torch.manual_seed(self.seed)
                torch.cuda.manual_seed(self.seed)

            if not hasattr(self, "bivae"):
                num_items = train_set.matrix.shape[1]
                num_users = train_set.matrix.shape[0]
                self.bivae = BiVAE(
                    k=self.k,
                    user_encoder_structure=[num_items] + self.user_encoder_structure,
                    item_encoder_structure=[num_users] + self.item_encoder_structure,
                    act_fn=self.act_fn,
                    likelihood=self.likelihood,
                    cap_priors=self.cap_priors,
                    feature_dim=feature_dim,
                    batch_size=self.batch_size,
                ).to(self.device)

            learn(
                self.bivae,
                train_set,
                n_epochs=self.n_epochs,
                batch_size=self.batch_size,
                learn_rate=self.learning_rate,
                beta_kl=self.beta_kl,
                verbose=self.verbose,
                device=self.device,
            )
        elif self.verbose:
            print("%s is trained already (trainable = False)" % (self.name))

        return self


    def score(self, user_idx, item_idx=None):
        """Predict the scores/ratings of a user for an item.

        Parameters
        ----------
        user_idx: int, required
            The index of the user for whom to perform score prediction.

        item_idx: int, optional, default: None
            The index of the item for which to perform score prediction.
            If None, scores for all known items will be returned.

        Returns
        -------
        res : A scalar or a Numpy array
            Relative scores that the user gives to the item or to all known items

        """
        if self.is_unknown_user(user_idx):
            raise ScoreException("Can't make score prediction for user %d" % user_idx)

        if item_idx is not None and self.is_unknown_item(item_idx):
            raise ScoreException("Can't make score prediction for item %d" % item_idx)

        if item_idx is None:
            theta_u = self.bivae.mu_theta[user_idx].view(1, -1)
            beta = self.bivae.mu_beta
            return self.bivae.decode_user(theta_u, beta).cpu().numpy().ravel()
        else:
            theta_u = self.bivae.mu_theta[user_idx].view(1, -1)
            beta_i = self.bivae.mu_beta[item_idx].view(1, -1)
            pred = self.bivae.decode_user(theta_u, beta_i).cpu().numpy().ravel()
            return scale(pred, self.min_rating, self.max_rating, 0.0, 1.0)


    def get_vector_measure(self):
        """Getting a valid choice of vector measurement in ANNMixin._measures.

        Returns
        -------
        measure: MEASURE_DOT
            Dot product aka. inner product
        """
        return MEASURE_DOT


    def get_user_vectors(self):
        """Getting a matrix of user vectors serving as query for ANN search.

        Returns
        -------
        out: numpy.array
            Matrix of user vectors for all users available in the model.
        """
        user_vectors = self.bivae.mu_theta.detach().cpu().numpy()
        return user_vectors


    def get_item_vectors(self):
        """Getting a matrix of item vectors used for building the index for ANN search.

        Returns
        -------
        out: numpy.array
            Matrix of item vectors for all items available in the model.
        """
        item_vectors = self.bivae.mu_beta.detach().cpu().numpy()
        return item_vectors



# Functions

In [10]:
def predict_ranking(
    model,
    data,
    usercol='userID',
    itemcol='itemID',
    predcol='pred',
    remove_seen=False,
):
    """Computes predictions of recommender model from Cornac on all users and items in data.
    It can be used for computing ranking metrics like NDCG.

    Args:
        model (cornac.models.Recommender): A recommender model from Cornac
        data (pandas.DataFrame): The data from which to get the users and items
        usercol (str): Name of the user column
        itemcol (str): Name of the item column
        remove_seen (bool): Flag to remove (user, item) pairs seen in the training data

    Returns:
        pandas.DataFrame: Dataframe with usercol, itemcol, predcol
    """
    users, items, preds = [], [], []
    item = list(model.iid_map.keys())
    for uid, user_idx in model.uid_map.items():
        user = [uid] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(model.score(user_idx).tolist())

    all_predictions = pd.DataFrame(
        data={usercol: users, itemcol: items, predcol: preds}
    )

    if remove_seen:
        tempdf = pd.concat(
            [
                data[[usercol, itemcol]],
                pd.DataFrame(
                    data=np.ones(data.shape[0]), columns=["dummycol"], index=data.index
                ),
            ],
            axis=1,
        )
        merged = pd.merge(tempdf, all_predictions, on=[usercol, itemcol], how="outer")
        return merged[merged["dummycol"].isnull()].drop("dummycol", axis=1)
    else:
        return all_predictions

In [11]:
def predict_ranking_user(
    model,
    data,
    user_id,
    usercol='userID',
    itemcol='itemID',
    predcol='pred',
    remove_seen=False,
):
    """Computes predictions of recommender model from Cornac on all users and items in data.
    It can be used for computing ranking metrics like NDCG.

    Args:
        model (cornac.models.Recommender): A recommender model from Cornac
        data (pandas.DataFrame): The data from which to get the users and items
        usercol (str): Name of the user column
        itemcol (str): Name of the item column
        remove_seen (bool): Flag to remove (user, item) pairs seen in the training data

    Returns:
        pandas.DataFrame: Dataframe with usercol, itemcol, predcol
    """
    users, items, preds = [], [], []
    item = list(model.iid_map.keys())
    # reverse_uid_map = {val : key for key,val in dict(model_.uid_map).items()}
    user_idx = dict(model_.uid_map).get(user_id)

    user_data = data.loc[data[usercol] == user_id]

    # user = [uid] * len(item)
    # users.extend(user)
    items.extend(item)
    preds.extend(model.score(user_idx).tolist())

    all_predictions = pd.DataFrame(
        data={itemcol: items, predcol: preds}
    )

    if remove_seen:
        tempdf = pd.concat(
            [
                user_data[[itemcol]],
                pd.DataFrame(
                    data=np.ones(user_data.shape[0]), columns=["dummycol"], index=user_data.index
                ),
            ],
            axis=1,
        )
        merged = pd.merge(tempdf, all_predictions, on=[itemcol], how="outer")
        return merged[merged["dummycol"].isnull()].drop("dummycol", axis=1)
    else:
        return all_predictions

# Insight generation

In [12]:
data_path = '../../data/phase_2/price_data_v2.xlsx'

data = pd.read_excel(data_path, index_col= False)

In [13]:
data.head()


,SECURITYCODE,OPENINGPRICE,HIGHPX,LOWPX,CLOSINGPRICE,TRADEDATE,UNIX_TS
0,EXT,7.3,7.5,7.3,7.5,2024-02-07,1707264000
1,COOP,2.3,2.3,2.2,2.2,2024-02-07,1707264000
2,MDL,8.3,8.6,8.3,8.6,2024-02-07,1707264000
3,FCT,24.9,26.4,24.8,26.2,2024-02-07,1707264000
4,HBS,11.5,11.6,11.5,11.5,2024-02-07,1707264000


# Plot OHLC

In [14]:
def plot_ohlc(data, title):
    x_vals = data.TRADEDATE

    fig = make_subplots(rows=1, cols=1, vertical_spacing=0.01 , specs=[[{'rowspan':1, 'type':'Candlestick'}]],shared_xaxes=True)
    ohlc_obj = go.Candlestick(x=x_vals, open = data.OPENINGPRICE, high=data.HIGHPX, low=data.LOWPX, close=data.CLOSINGPRICE, name= title)
    fig.add_trace(ohlc_obj, row = 1, col = 1)
    fig.update_layout(title = title)
    fig.show(config={
        'modeBarButtonsToRemove': ['zoom', 'pan']
    })

In [15]:
# from turtle import bgcolor
# from matplotlib.axis import XAxis


def plot_ohlc_w_des(data, titile, des):
    x_vals = data.TRADEDATE

    fig = make_subplots(rows=1, cols=2, vertical_spacing=0.01, column_widths=[0.15, 0.85], subplot_titles=["", "Chart"]) #,shared_xaxes=True
    ohlc_obj = go.Candlestick(x=x_vals, open = data.OPENINGPRICE, high=data.HIGHPX, low=data.LOWPX, close=data.CLOSINGPRICE, name= 'OHLC')
    fig.add_trace(ohlc_obj, row = 1, col = 2)
    fig.add_trace(go.Scatter(x=[], y=[]), row=1, col=1)

    fig.add_annotation(x=10, y=10, xref="paper", yref="paper",
                   text=des, showarrow=False,
                   align = 'left',
                   font=dict(size=10, color="black"),
                   row=1, col=1)

    fig.update_layout(xaxis=dict(visible=False), yaxis=dict(visible=False))
    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, row=1, col=1)
    fig.update_annotations(selector=dict(row=1, col=1), paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(height=500, width=1500, title_text=titile)

    return fig

In [16]:
# from turtle import bgcolor
# from matplotlib.axis import XAxis


def plot_ohlc_v1(data):
    x_vals = data.TRADEDATE

    fig = make_subplots(rows=1, cols=1, vertical_spacing=0.01, specs=[[{'rowspan':1, 'type':'Candlestick'}]])
    ohlc_obj = go.Candlestick(x=x_vals, open = data.OPENINGPRICE, high=data.HIGHPX, low=data.LOWPX, close=data.CLOSINGPRICE, name= 'OHLC')
    fig.add_trace(ohlc_obj, row = 1, col = 1)

    # fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, row=1, col=1)
    fig.update_layout(height=450, width=995)

    return fig

## OHLC candles

## Volatility calculations

In [17]:
def get_log_returns_v2(df ,close_col , horizon):
  data1 = df[close_col][horizon:].to_numpy()
  data2 = df[close_col][:-horizon].to_numpy()
  df = df.reset_index(drop = True)
  df.loc[horizon:, 'returns'] = data1/data2
  df['log_returns'] = np.log(df['returns'])
  df = df.dropna(axis = 0, how = 'any').reset_index(drop = True)
  return df

In [18]:
def get_std_volatility(df, close_col = 'CLOSINGPRICE', horizon = 5):

    df_out = get_log_returns_v2(df, 'CLOSINGPRICE', 5)
    return_mean = df_out.log_returns.mean()
    return_std = df_out.log_returns.std()

    return return_mean, return_std

### Garman Klass volatility
![image.png](attachment:image.png)

In [19]:
def GKHV(o,h,l,c):

    volatility = (1/2)*((np.log(h/l))**2) + (2*(np.log(2))-1)*((np.log(c/o))**2)

    return volatility

### Rogers and Satchell

![image.png](attachment:image.png)

In [20]:
def RS(o,h,l,c,prev_c):

    u = np.log(h/o)
    c = np.log(c/o)
    d = np.log(l/o)


    volatility = u*(u-c) + d*(d-c)

    return volatility


### Yand and Zhang
![image.png](attachment:image.png)

In [21]:
symbol_volatilities = pd.DataFrame()
v_dict = {}
symbols = data
for symbol in symbols:
    # print(symbol)
    v_dict['symbol'] = symbol

    symbol_df = data.loc[data.SECURITYCODE == symbol].sort_values(by = 'UNIX_TS').reset_index(drop = True)
    
    symbol_df['gkhv'] = symbol_df.apply(lambda row: GKHV(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE']), axis = 1)
    gkhv = np.sqrt(symbol_df.gkhv.mean())
    v_dict['gkhv'] = gkhv

    symbol_df['prev_c'] = symbol_df['CLOSINGPRICE'].shift(1)
    symbol_df['rs'] = symbol_df.apply(lambda row: RS(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE'], row['prev_c']), axis = 1)
    rs = np.sqrt(symbol_df.rs.mean())
    v_dict['rs'] = rs

    symbol_df['norm_o'] = symbol_df['OPENINGPRICE']/symbol_df['CLOSINGPRICE'].shift(1)
    symbol_df['norm_c'] = symbol_df['CLOSINGPRICE']/symbol_df['OPENINGPRICE']
    k = (0.34/(1.34 + ((len(symbol_df)+1)/(len(symbol_df)-1))))
    yangzhang = np.sqrt((symbol_df['norm_o'].std()**2) + (k*symbol_df['norm_c'].std()**2) + (1-k)*symbol_df['rs'].mean())
    v_dict['yangzhang'] = yangzhang

    symbol_volatilities = pd.concat([symbol_volatilities, pd.DataFrame([v_dict])], ignore_index = True)

### get_volatlity_insights

In [22]:
def get_volatility_insights(symbol_df):

    symbol_df['gkhv'] = symbol_df.apply(lambda row: GKHV(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE']), axis = 1)
    gkhv = (np.sqrt(symbol_df.gkhv.mean()))*100

    symbol_df['prev_c'] = symbol_df['CLOSINGPRICE'].shift(1)
    symbol_df['rs'] = symbol_df.apply(lambda row: RS(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE'], row['prev_c']), axis = 1)
    rs = (np.sqrt(symbol_df.rs.mean()))*100

    symbol_df['norm_o'] = symbol_df['OPENINGPRICE']/symbol_df['CLOSINGPRICE'].shift(1)
    symbol_df['norm_c'] = symbol_df['CLOSINGPRICE']/symbol_df['OPENINGPRICE']
    k = (0.34/(1.34 + ((len(symbol_df)+1)/(len(symbol_df)-1))))
    
    yangzhang = np.sqrt((symbol_df['norm_o'].std()**2) + (k*symbol_df['norm_c'].std()**2) + (1-k)*symbol_df['rs'].mean())*100
    
    return gkhv, rs, yangzhang

## Drowdown

In [23]:
# returns percentage
def max_drowdown(symb_df,close):
    max_prs = close.rolling(window = len(symb_df), min_periods = 1).max()
    dd = (close/max_prs) - 1
    max_dd = dd.rolling(window = len(dd), min_periods=1).min()
    return max_dd.min()*100


## Returns

In [24]:
def find_act_fday(symbol_df ,possible_dates):

    for pos_date in possible_dates:
    # print(pos_date)
        if pos_date in symbol_df.TRADEDATE.values:
            return pos_date
            # break
    
    # return pos_date

### get_yr_2_date

In [25]:
#returns the percentage value
def get_yr_2_date(symbol_df):
    # fday = pd.Timestamp(datetime((pd.Timestamp.today().year),1,1).date())

    # fixing current date as per the dataset. ideally should be the actual current date
    fday = pd.Timestamp(datetime((symbol_df.TRADEDATE.max().year),1,1).date())

    possible_dates = pd.bdate_range(fday, fday+pd.offsets.BusinessDay(n=5))

    act_fday = find_act_fday(symbol_df,possible_dates)
    
    fday_value = symbol_df.loc[symbol_df.TRADEDATE == act_fday].CLOSINGPRICE.iloc[0]
    today_value = symbol_df.iloc[-1].CLOSINGPRICE

    yr_2_dt_return = (today_value - fday_value)/fday_value

    yr_2_date_df = symbol_df[symbol_df.TRADEDATE >= act_fday].sort_values(by = 'UNIX_TS')
    gkhv, rs, yangzhang = get_volatility_insights(yr_2_date_df)

    max_dd = max_drowdown(yr_2_date_df, yr_2_date_df['CLOSINGPRICE'])

    # return yr_2_dt_return*100, max_dd, gkhv, rs, yangzhang
    return round(yr_2_dt_return*100, 2), round(max_dd, 2) , round(gkhv, 2), round(rs, 2), round(yangzhang, 2)

### get_yr

In [26]:
# returns the percentage value
def get_yr(symbol_df):

    fday = symbol_df.TRADEDATE.iloc[-1] - pd.DateOffset(years=1)
    possible_dates = pd.bdate_range(fday, fday+pd.offsets.BusinessDay(n=20))
    
    act_fday = find_act_fday(symbol_df,possible_dates)
    
    fday_value = symbol_df.loc[symbol_df.TRADEDATE == act_fday].CLOSINGPRICE.iloc[0]
    today_value = symbol_df.iloc[-1].CLOSINGPRICE

    yr_return = (today_value - fday_value)/fday_value

    yr_df = symbol_df[symbol_df.TRADEDATE >= act_fday].sort_values(by = 'UNIX_TS')
    gkhv, rs, yangzhang = get_volatility_insights(yr_df)

    max_dd = max_drowdown(yr_df, yr_df['CLOSINGPRICE'])

    # return yr_return*100, max_dd, gkhv, rs, yangzhang
    return round(yr_return*100, 2), round(max_dd, 2) , round(gkhv, 2), round(rs, 2), round(yangzhang, 2)

### get_mn

In [27]:
# returns the percentage value
def get_mn(symbol_df):

    fday = symbol_df.TRADEDATE.iloc[-1] - pd.DateOffset(months=1)
    possible_dates = pd.bdate_range(fday, fday+pd.offsets.BusinessDay(n=5))

    act_fday = find_act_fday(symbol_df,possible_dates)
    
    fday_value = symbol_df.loc[symbol_df.TRADEDATE == act_fday].CLOSINGPRICE.iloc[0]
    today_value = symbol_df.iloc[-1].CLOSINGPRICE

    mn_return = (today_value - fday_value)/fday_value

    mn_df = symbol_df[symbol_df.TRADEDATE >= act_fday].sort_values(by = 'UNIX_TS')
    gkhv, rs, yangzhang = get_volatility_insights(mn_df)

    max_dd = max_drowdown(mn_df, mn_df['CLOSINGPRICE'])

    return round(mn_return*100, 2), round(max_dd, 2) , round(gkhv, 2), round(rs, 2), round(yangzhang, 2)

In [28]:
def get_insights(symbol):
    insight_dict = {'year_to_date':{}, 'last_year':{}, 'last_month' : {}}
    symbol_df = data.loc[data.SECURITYCODE == symbol].sort_values(by = 'UNIX_TS')

    insight_dict['year_to_date']['yr_2_dt_return'], insight_dict['year_to_date']['yr_2_dt_mx_dd'], insight_dict['year_to_date']['gkhv'], insight_dict['year_to_date']['rs'], insight_dict['year_to_date']['yangzhang'] = get_yr_2_date(symbol_df)
    insight_dict['last_year']['yr_2_dt_return'], insight_dict['last_year']['yr_2_dt_mx_dd'], insight_dict['last_year']['gkhv'], insight_dict['last_year']['rs'], insight_dict['last_year']['yangzhang'] = get_yr(symbol_df)
    insight_dict['last_month']['yr_2_dt_return'], insight_dict['last_month']['yr_2_dt_mx_dd'], insight_dict['last_month']['gkhv'], insight_dict['last_month']['rs'], insight_dict['last_month']['yangzhang'] = get_mn(symbol_df)

    return insight_dict

In [29]:
def get_insights_html(symbol):
    insight_dict = {'year_to_date':{}, 'last_year':{}, 'last_month' : {}}
    symbol_df = data.loc[data.SECURITYCODE == symbol].sort_values(by = 'UNIX_TS')

    y2d = get_yr_2_date(symbol_df)
    yr = get_yr(symbol_df)
    mn = get_mn(symbol_df)

    desc = 'year to date : <br>'\
    '   return : {}%<br>'\
    '   max drow down : {}%<br>'\
    '   gkhv : {}%<br>'\
    '   rs : {}%<br>'\
    '   yangzhang : {}%<br>'\
    ' <br>'\
    'last year : <br>'\
    '   return : {}%<br>'\
    '   max drow down : {}%<br>'\
    '   gkhv : {}%<br>'\
    '   rs : {}%<br>'\
    '   yangzhang : {}%<br>'\
    ' <br>'\
    'last month : <br>'\
    '   return : {}%<br>'\
    '   max drow down : {}%<br>'\
    '   gkhv : {}%<br>'\
    '   rs : {}%<br>'\
    '   yangzhang : {}%<br>'.format(y2d[0],y2d[1],y2d[2],y2d[3],y2d[4],
                        yr[0],yr[1],yr[2],yr[3],yr[4],
                        mn[0],mn[1],mn[2],mn[3],mn[4])
    return desc

In [30]:
def get_insights_str(symbol):
    # insight_dict = {'year_to_date':{}, 'last_year':{}, 'last_month' : {}}
    symbol_df = data.loc[data.SECURITYCODE == symbol].sort_values(by = 'UNIX_TS')

    y2d = get_yr_2_date(symbol_df)
    yr = get_yr(symbol_df)
    mn = get_mn(symbol_df)
  
    return ([y2d[0],y2d[1],y2d[2],y2d[3],y2d[4],
                        yr[0],yr[1],yr[2],yr[3],yr[4],
                        mn[0],mn[1],mn[2],mn[3],mn[4]])

# Combined Recommender

In [31]:
price_data = data.copy()
price_data.head(2)

,SECURITYCODE,OPENINGPRICE,HIGHPX,LOWPX,CLOSINGPRICE,TRADEDATE,UNIX_TS
0,EXT,7.3,7.5,7.3,7.5,2024-02-07,1707264000
1,COOP,2.3,2.3,2.2,2.2,2024-02-07,1707264000


In [32]:
# save_loc = r'../models\BiVAECF_alldata_24_02_16_15_32\BiVAECF\2024-02-16_15-32-24-532947.pkl'
save_loc = r'..\..\models\BiVAECF_alldata_24_02_16_15_32\BiVAECF\2024-02-16_15-32-24-532947.pkl'
model_ = BiVAECF.load(save_loc)

In [33]:
stock_data_excel_file = pd.ExcelFile('../../data/phase_2/stock_data.xlsx')
data_path = '../../data/phase_2/price_data_v2.xlsx'

data = pd.read_excel(data_path, index_col= False)
price_symbols = list(data.SECURITYCODE.unique())

stock_data = pd.read_excel(stock_data_excel_file, 'phase 1 symbols reduced')

stock_data = stock_data[stock_data.Symbol.isin(price_symbols)].copy()

stock_data['Symbol'] = stock_data['Symbol'].str.replace(r"(","")
stock_data['Symbol'] = stock_data['Symbol'].str.replace(r")","")

embeddings = HuggingFaceEmbeddings()
stock_meta_data = {stock_data.iloc[idx]['Symbol'] : stock_data.iloc[idx]['Name'] for idx in range(stock_data.shape[0])}


C:\Users\naradaw\AppData\Local\Temp\ipykernel_17212\432685352.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stock_data['Symbol'] = stock_data['Symbol'].str.replace(r"(","")
C:\Users\naradaw\AppData\Local\Temp\ipykernel_17212\432685352.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stock_data['Symbol'] = stock_data['Symbol'].str.replace(r")","")


# gradio fn

In [48]:
def quey_generator(domains):
    # query = 'busines in the domain of '+','.join(domains)
    query = ','.join(domains)
    return query

####
def get_cf_recos(
        symbols, 
        model_ = model_,
        itemcol='itemID',
        predcol='pred',
        remove_seen=False,
        top_k = 10,
        val_symbols = price_symbols
        ):


    user_symbol_idx = np.array([model_.iid_map.get(symbol) for symbol in symbols])
    user_vec = np.zeros(len(model_.iid_map.keys()))
    user_vec[user_symbol_idx] =1
    
    user_encoder = model_.bivae.user_encoder

    user_mu = model_.bivae.user_mu
    user_ten = torch.from_numpy(user_vec)
    user_ten = user_ten.to("cuda:0").float()

    user_lat_vec = user_mu(user_encoder(user_ten)).view(1,-1)

    beta_mu = model_.bivae.mu_beta

    preds = model_.bivae.decode_user(user_lat_vec, beta_mu)
    preds = preds.detach().cpu().numpy().ravel()
    
    items = list(model_.iid_map.keys())
    # all_predictions = pd.DataFrame(data = {itemcol:items})
    

    all_preds = pd.DataFrame(
        data={itemcol: items, predcol: preds}
    )
    if remove_seen:
        tempdf = pd.concat(
            [
                pd.DataFrame(data = symbols, columns = [itemcol]), 
                pd.DataFrame(
                    data=np.ones(len(symbols)), columns=["dummycol"]
                ),
            ],
            axis=1,
        )
        merged = pd.merge(tempdf, all_preds, on=[itemcol], how="outer")
        all_preds = merged[merged["dummycol"].isnull()].drop("dummycol", axis=1)
        # this is temp. need to only out put symbols of which we have price data to generate insights
        all_preds_fil = all_preds[all_preds.itemID.isin(val_symbols)].copy()
        top_k_symbols = list(all_preds_fil.sort_values(by = 'pred', ascending= False).itemID.iloc[:top_k].values)
        return top_k_symbols
    else:
        all_preds_fil = all_preds[all_preds.itemID.isin(val_symbols)].copy()
        top_k_symbols = list(all_preds_fil.sort_values(by = 'pred', ascending= False).itemID.iloc[:top_k].values)
        return top_k_symbols
    

####
def get_recommendations_gradio_cf(domains, top_k = 5):
    query = quey_generator(domains)
    relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
    recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name'), relevant_stocks[idx][1]) for idx in range(len(relevant_stocks))]

    # recommended_symbols = []
    recommended_symbols = [data[0] for data in recommendations]
    cf_recos = get_cf_recos(recommended_symbols)
    cf_recommendations = [(symbol, stock_meta_data.get(symbol)) for symbol in cf_recos]


    reco_symbols = []
    reco_insights = []
    reco_plots = []

    for data in cf_recommendations:
        print(data[0], data[1])
        # recommended_symbols.append(data[0])
        symbol_df = price_data[price_data.SECURITYCODE == data[0]].sort_values(by = 'UNIX_TS').reset_index(drop = True)
        
        reco_symbol = '{} : {}'.format(data[0], data[1])
        reco_acc = gr.Accordion(reco_symbol)
        reco_symbols.append(reco_acc)

        reco_insight = get_insights_str(data[0])
        reco_insights = reco_insights + reco_insight

        reco_plot = plot_ohlc_v1(symbol_df)
        reco_grplot = gr.Plot(reco_plot, scale = 1, min_width= 1000)
        reco_plots.append(reco_grplot)

    return reco_symbols + reco_plots + reco_insights

def get_recommendations_gradio_gr_v3(domains, reco_type, top_k = 10):
    query = quey_generator(domains)
    relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
    recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name'), relevant_stocks[idx][1]) for idx in range(len(relevant_stocks))]

    cb_recos = [(data[0], data[1]) for data in recommendations]
    recommended_symbols = [data[0] for data in recommendations]
    cf_recos = get_cf_recos(recommended_symbols)
    cf_recommendations = [(symbol, stock_meta_data.get(symbol)) for symbol in cf_recos]


    reco_symbols = []
    reco_insights = []
    reco_plots = []

    print('reco_type : \n', reco_type)
    print('domains : \n', domains)
    # print(len(cb_recos))
    if reco_type == 'Content Base':
        for data in cb_recos:
            print(data[0], data[1])
            symbol_df = price_data[price_data.SECURITYCODE == data[0]].sort_values(by = 'UNIX_TS').reset_index(drop = True)
            
            reco_symbol = '{} : {}'.format(data[0], data[1])
            reco_acc = gr.Accordion(reco_symbol)
            reco_symbols.append(reco_acc)

            reco_insight = get_insights_str(data[0])
            reco_insights = reco_insights + reco_insight

            reco_plot = plot_ohlc_v1(symbol_df)
            reco_grplot = gr.Plot(reco_plot, scale = 1, min_width= 1000)
            reco_plots.append(reco_grplot)
        print(len(reco_symbols + reco_plots + reco_insights))
        return reco_symbols + reco_plots + reco_insights
    elif reco_type == 'Collaborative Filtering':
        for data in cf_recommendations:
            print(data[0], data[1])
            symbol_df = price_data[price_data.SECURITYCODE == data[0]].sort_values(by = 'UNIX_TS').reset_index(drop = True)
            
            reco_symbol = '{} : {}'.format(data[0], data[1])
            reco_acc = gr.Accordion(reco_symbol)
            reco_symbols.append(reco_acc)

            reco_insight = get_insights_str(data[0])
            reco_insights = reco_insights + reco_insight

            reco_plot = plot_ohlc_v1(symbol_df)
            reco_grplot = gr.Plot(reco_plot, scale = 1, min_width= 1000)
            reco_plots.append(reco_grplot)
        return reco_symbols + reco_plots + reco_insights

# gradio ui - me no like :(

In [53]:
%%blocks

with gr.Blocks() as demo:
    dd_input = gr.Dropdown(
        ["finance", "transport", "technology", "agriculture"],
        multiselect=True,
        label="Domains",
        allow_custom_value=True,
        scale=5
    )
    # ttxt_input = gr.Textbox(
    #     placeholder = 'What kind of business do you prefer?'
    # )

    rad_btn = gr.Radio(
        choices = ['Content Base','Collaborative Filtering'],
        label = 'recommender type',
        value = 'Content Base'
    )
    button = gr.Button("Generate Recommendations")

    # with gr.Tab(label = 'Collaborative Filtering'):
    with gr.Column() as output_col:
        # 1
        with gr.Accordion(open=False) as acc1:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in1_1 = gr.Textbox(label='Return')
                        in1_2 = gr.Textbox(label='max drow down')
                        in1_3 = gr.Textbox(label='gkhv')
                        in1_4 = gr.Textbox(label='rs')
                        in1_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in1_6 = gr.Textbox(label='Return')
                        in1_7 = gr.Textbox(label='max drow down')
                        in1_8 = gr.Textbox(label='gkhv')
                        in1_9 = gr.Textbox(label='rs')
                        in1_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in1_11 = gr.Textbox(label='Return')
                        in1_12 = gr.Textbox(label='max drow down')
                        in1_13 = gr.Textbox(label='gkhv')
                        in1_14 = gr.Textbox(label='rs')
                        in1_15 = gr.Textbox(label='yangzhang')
                plt1 = gr.Plot()
        # 2
        with gr.Accordion(open=False) as acc2:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in2_1 = gr.Textbox(label='Return')
                        in2_2 = gr.Textbox(label='max drow down')
                        in2_3 = gr.Textbox(label='gkhv')
                        in2_4 = gr.Textbox(label='rs')
                        in2_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in2_6 = gr.Textbox(label='Return')
                        in2_7 = gr.Textbox(label='max drow down')
                        in2_8 = gr.Textbox(label='gkhv')
                        in2_9 = gr.Textbox(label='rs')
                        in2_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in2_11 = gr.Textbox(label='Return')
                        in2_12 = gr.Textbox(label='max drow down')
                        in2_13 = gr.Textbox(label='gkhv')
                        in2_14 = gr.Textbox(label='rs')
                        in2_15 = gr.Textbox(label='yangzhang')
                plt2 = gr.Plot()
        # 3
        with gr.Accordion(open=False) as acc3:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in3_1 = gr.Textbox(label='Return')
                        in3_2 = gr.Textbox(label='max drow down')
                        in3_3 = gr.Textbox(label='gkhv')
                        in3_4 = gr.Textbox(label='rs')
                        in3_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in3_6 = gr.Textbox(label='Return')
                        in3_7 = gr.Textbox(label='max drow down')
                        in3_8 = gr.Textbox(label='gkhv')
                        in3_9 = gr.Textbox(label='rs')
                        in3_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in3_11 = gr.Textbox(label='Return')
                        in3_12 = gr.Textbox(label='max drow down')
                        in3_13 = gr.Textbox(label='gkhv')
                        in3_14 = gr.Textbox(label='rs')
                        in3_15 = gr.Textbox(label='yangzhang')
                plt3 = gr.Plot()
        #4
        with gr.Accordion(open=False) as acc4:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in4_1 = gr.Textbox(label='Return')
                        in4_2 = gr.Textbox(label='max drow down')
                        in4_3 = gr.Textbox(label='gkhv')
                        in4_4 = gr.Textbox(label='rs')
                        in4_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in4_6 = gr.Textbox(label='Return')
                        in4_7 = gr.Textbox(label='max drow down')
                        in4_8 = gr.Textbox(label='gkhv')
                        in4_9 = gr.Textbox(label='rs')
                        in4_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in4_11 = gr.Textbox(label='Return')
                        in4_12 = gr.Textbox(label='max drow down')
                        in4_13 = gr.Textbox(label='gkhv')
                        in4_14 = gr.Textbox(label='rs')
                        in4_15 = gr.Textbox(label='yangzhang')
                plt4 = gr.Plot()
        
        # 5
        with gr.Accordion(open=False) as acc5:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in5_1 = gr.Textbox(label='Return')
                        in5_2 = gr.Textbox(label='max drow down')
                        in5_3 = gr.Textbox(label='gkhv')
                        in5_4 = gr.Textbox(label='rs')
                        in5_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in5_6 = gr.Textbox(label='Return')
                        in5_7 = gr.Textbox(label='max drow down')
                        in5_8 = gr.Textbox(label='gkhv')
                        in5_9 = gr.Textbox(label='rs')
                        in5_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in5_11 = gr.Textbox(label='Return')
                        in5_12 = gr.Textbox(label='max drow down')
                        in5_13 = gr.Textbox(label='gkhv')
                        in5_14 = gr.Textbox(label='rs')
                        in5_15 = gr.Textbox(label='yangzhang')
                plt5 = gr.Plot()

        # 6
        with gr.Accordion(open=False) as acc6:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in6_1 = gr.Textbox(label='Return')
                        in6_2 = gr.Textbox(label='max drow down')
                        in6_3 = gr.Textbox(label='gkhv')
                        in6_4 = gr.Textbox(label='rs')
                        in6_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in6_6 = gr.Textbox(label='Return')
                        in6_7 = gr.Textbox(label='max drow down')
                        in6_8 = gr.Textbox(label='gkhv')
                        in6_9 = gr.Textbox(label='rs')
                        in6_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in6_11 = gr.Textbox(label='Return')
                        in6_12 = gr.Textbox(label='max drow down')
                        in6_13 = gr.Textbox(label='gkhv')
                        in6_14 = gr.Textbox(label='rs')
                        in6_15 = gr.Textbox(label='yangzhang')
                plt6 = gr.Plot()
            #7
        with gr.Accordion(open=False) as acc7:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in7_1 = gr.Textbox(label='Return')
                        in7_2 = gr.Textbox(label='max drow down')
                        in7_3 = gr.Textbox(label='gkhv')
                        in7_4 = gr.Textbox(label='rs')
                        in7_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in7_6 = gr.Textbox(label='Return')
                        in7_7 = gr.Textbox(label='max drow down')
                        in7_8 = gr.Textbox(label='gkhv')
                        in7_9 = gr.Textbox(label='rs')
                        in7_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in7_11 = gr.Textbox(label='Return')
                        in7_12 = gr.Textbox(label='max drow down')
                        in7_13 = gr.Textbox(label='gkhv')
                        in7_14 = gr.Textbox(label='rs')
                        in7_15 = gr.Textbox(label='yangzhang')
                plt7 = gr.Plot()
        
        # 8
        with gr.Accordion(open=False) as acc8:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in8_1 = gr.Textbox(label='Return')
                        in8_2 = gr.Textbox(label='max drow down')
                        in8_3 = gr.Textbox(label='gkhv')
                        in8_4 = gr.Textbox(label='rs')
                        in8_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in8_6 = gr.Textbox(label='Return')
                        in8_7 = gr.Textbox(label='max drow down')
                        in8_8 = gr.Textbox(label='gkhv')
                        in8_9 = gr.Textbox(label='rs')
                        in8_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in8_11 = gr.Textbox(label='Return')
                        in8_12 = gr.Textbox(label='max drow down')
                        in8_13 = gr.Textbox(label='gkhv')
                        in8_14 = gr.Textbox(label='rs')
                        in8_15 = gr.Textbox(label='yangzhang')
                plt8 = gr.Plot()

        # 9
        with gr.Accordion(open=False) as acc9:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in9_1 = gr.Textbox(label='Return')
                        in9_2 = gr.Textbox(label='max drow down')
                        in9_3 = gr.Textbox(label='gkhv')
                        in9_4 = gr.Textbox(label='rs')
                        in9_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in9_6 = gr.Textbox(label='Return')
                        in9_7 = gr.Textbox(label='max drow down')
                        in9_8 = gr.Textbox(label='gkhv')
                        in9_9 = gr.Textbox(label='rs')
                        in9_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in9_11 = gr.Textbox(label='Return')
                        in9_12 = gr.Textbox(label='max drow down')
                        in9_13 = gr.Textbox(label='gkhv')
                        in9_14 = gr.Textbox(label='rs')
                        in9_15 = gr.Textbox(label='yangzhang')
                plt9 = gr.Plot()

        with gr.Accordion(open=False) as acc10:
            with gr.Row():
                with gr.Tab(label='Year to Date'):
                    with gr.Column():
                        in10_1 = gr.Textbox(label='Return')
                        in10_2 = gr.Textbox(label='max drow down')
                        in10_3 = gr.Textbox(label='gkhv')
                        in10_4 = gr.Textbox(label='rs')
                        in10_5 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Year'):
                    with gr.Column():
                        in10_6 = gr.Textbox(label='Return')
                        in10_7 = gr.Textbox(label='max drow down')
                        in10_8 = gr.Textbox(label='gkhv')
                        in10_9 = gr.Textbox(label='rs')
                        in10_10 = gr.Textbox(label='yangzhang')
                with gr.Tab(label='Last Month'):
                    with gr.Column():
                        in10_11 = gr.Textbox(label='Return')
                        in10_12 = gr.Textbox(label='max drow down')
                        in10_13 = gr.Textbox(label='gkhv')
                        in10_14 = gr.Textbox(label='rs')
                        in10_15 = gr.Textbox(label='yangzhang')
                plt10 = gr.Plot()
    outputs = [acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,
            plt1,plt2,plt3,plt4,plt5,plt6,plt7,plt8,plt9,plt10,
            in1_1, in1_2, in1_3, in1_4, in1_5, 
            in1_6, in1_7, in1_8, in1_9, in1_10, 
            in1_11, in1_12, in1_13, in1_14, in1_15, 
            in2_1, in2_2, in2_3, in2_4, in2_5, 
            in2_6, in2_7, in2_8, in2_9, in2_10, 
            in2_11, in2_12, in2_13, in2_14, in2_15, 
            in3_1, in3_2, in3_3, in3_4, in3_5, 
            in3_6, in3_7, in3_8, in3_9, in3_10, 
            in3_11, in3_12, in3_13, in3_14, in3_15, 
            in4_1, in4_2, in4_3, in4_4, in4_5, 
            in4_6, in4_7, in4_8, in4_9, in4_10, 
            in4_11, in4_12, in4_13, in4_14, in4_15, 
            in5_1, in5_2, in5_3, in5_4, in5_5, 
            in5_6, in5_7, in5_8, in5_9, in5_10, 
            in5_11, in5_12, in5_13, in5_14, in5_15, 
            in6_1, in6_2, in6_3, in6_4, in6_5, 
            in6_6, in6_7, in6_8, in6_9, in6_10, 
            in6_11, in6_12, in6_13, in6_14, in6_15, 
            in7_1, in7_2, in7_3, in7_4, in7_5, 
            in7_6, in7_7, in7_8, in7_9, in7_10, 
            in7_11, in7_12, in7_13, in7_14, in7_15, 
            in8_1, in8_2, in8_3, in8_4, in8_5, 
            in8_6, in8_7, in8_8, in8_9, in8_10, 
            in8_11, in8_12, in8_13, in8_14, in8_15, 
            in9_1, in9_2, in9_3, in9_4, in9_5, 
            in9_6, in9_7, in9_8, in9_9, in9_10, 
            in9_11, in9_12, in9_13, in9_14, in9_15, 
            in10_1, in10_2, in10_3, in10_4, in10_5, 
            in10_6, in10_7, in10_8, in10_9, in10_10, 
            in10_11, in10_12, in10_13, in10_14, in10_15]
    inputs = [dd_input, rad_btn]
    
    button.click(fn = get_recommendations_gradio_gr_v3, inputs = inputs, outputs = outputs)
    rad_btn.change(fn = get_recommendations_gradio_gr_v3, inputs = inputs, outputs = outputs)

reco_type : 
 Content Base
domains : 
 ['finance', 'technology']
NTB NATIONS TRUST BANK PLC
LOFC LOLC FINANCE PLC
COMB COMMERCIAL BANK OF CEYLON PLC
JKH JOHN KEELLS HOLDINGS PLC
HNB HATTON NATIONAL BANK PLC
LPL LAUGFS POWER PLC
LCBF LANKA CREDIT AND BUSINESS FINANCE PLC
TAJ TAL LANKA HOTELS PLC
NDB NATIONAL DEVELOPMENT BANK PLC
FCT First Capital Treasuries PLC
170
reco_type : 
 Collaborative Filtering
domains : 
 ['finance', 'technology']
LOFC LOLC FINANCE PLC
EXPO EXPOLANKA HOLDINGS PLC
SAMP SAMPATH BANK PLC
RCL ROYAL CERAMICS LANKA PLC
COMB COMMERCIAL BANK OF CEYLON PLC
HAYL HAYLEYS PLC
LOLC L O L C HOLDINGS PLC
ACL ACL CABLES PLC
DIAL DIALOG AXIATA PLC
VONE VALLIBEL ONE PLC
reco_type : 
 Content Base
domains : 
 ['finance', 'technology']
NTB NATIONS TRUST BANK PLC
LOFC LOLC FINANCE PLC
COMB COMMERCIAL BANK OF CEYLON PLC
JKH JOHN KEELLS HOLDINGS PLC
HNB HATTON NATIONAL BANK PLC
LPL LAUGFS POWER PLC
LCBF LANKA CREDIT AND BUSINESS FINANCE PLC
TAJ TAL LANKA HOTELS PLC
NDB NATIONAL DEVEL

c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\langchain_core\vectorstores.py:311: UserWarning:

Relevance scores must be between 0 and 1, got [(Document(page_content='DISTILLERIES COMPANY OF SRI LANKA PLC distillation, manufacture and distribution of liquor products.', metadata={'symbol': 'DIST', 'name': 'DISTILLERIES COMPANY OF SRI LANKA PLC'}), -0.030631132271688344), (Document(page_content='EXPOLANKA HOLDINGS PLC diversified conglomerate company. It is specialized in logistics, leisure, and investments. It manages a portfolio of holdings, which consists of a range of diverse business operations and provides various function-based services to its companies. The logistics sector consists mainly of its freight forwarding business, represented by the EFL brand. It is\xa0engaged in providing air freight, ocean freight and other contract logistics services, such as warehousing and transport services. The sector also includes a GSA operation representing key strategic

reco_type : 
 Collaborative Filtering
domains : 
 ['Beverages', 'food']
EXPO EXPOLANKA HOLDINGS PLC
LIOC LANKA IOC PLC
ACL ACL CABLES PLC
LOFC LOLC FINANCE PLC
RCL ROYAL CERAMICS LANKA PLC
HAYL HAYLEYS PLC
LOLC L O L C HOLDINGS PLC
SAMP SAMPATH BANK PLC
CIC C I C HOLDINGS PLC
VONE VALLIBEL ONE PLC


c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\langchain_core\vectorstores.py:311: UserWarning:

Relevance scores must be between 0 and 1, got [(Document(page_content='DISTILLERIES COMPANY OF SRI LANKA PLC distillation, manufacture and distribution of liquor products.', metadata={'symbol': 'DIST', 'name': 'DISTILLERIES COMPANY OF SRI LANKA PLC'}), -0.030631132271688344), (Document(page_content='EXPOLANKA HOLDINGS PLC diversified conglomerate company. It is specialized in logistics, leisure, and investments. It manages a portfolio of holdings, which consists of a range of diverse business operations and provides various function-based services to its companies. The logistics sector consists mainly of its freight forwarding business, represented by the EFL brand. It is\xa0engaged in providing air freight, ocean freight and other contract logistics services, such as warehousing and transport services. The sector also includes a GSA operation representing key strategic

reco_type : 
 Content Base
domains : 
 ['Beverages', 'food']
DIST DISTILLERIES COMPANY OF SRI LANKA PLC
EXPO EXPOLANKA HOLDINGS PLC
TAJ TAL LANKA HOTELS PLC
HHL HEMAS HOLDINGS PLC
JKH JOHN KEELLS HOLDINGS PLC
JKL JOHN KEELLS PLC
DPL DANKOTUWA PORCELAIN PLC
JETS JETWING SYMPHONY PLC
LPL LAUGFS POWER PLC
CIC C I C HOLDINGS PLC
170


# gradio launch

In [37]:
# demo.launch(server_port = 7860)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# LOLLLLLLLL

In [38]:
sys.exit()

SystemExit: 

c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


10
EXPO EXPOLANKA HOLDINGS PLC
returned :  15
15
LOFC LOLC FINANCE PLC
returned :  15
30
RCL ROYAL CERAMICS LANKA PLC
returned :  15
45
ACL ACL CABLES PLC
returned :  15
60
LIOC LANKA IOC PLC
returned :  15
75
HAYL HAYLEYS PLC
returned :  15
90
SAMP SAMPATH BANK PLC
returned :  15
105
LOLC L O L C HOLDINGS PLC
returned :  15
120
VONE VALLIBEL ONE PLC
returned :  15
135
COMB COMMERCIAL BANK OF CEYLON PLC
returned :  15
150
170


c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\langchain_core\vectorstores.py:311: UserWarning:

Relevance scores must be between 0 and 1, got [(Document(page_content='HAYLEYS FIBRE PLC manufacture and export of coir fibre products. The Company is a total solutions provider for a range of applications based on coconut fibre and coir fibre in Sri Lanka. Its coconut fibre-related products and services include Bio-Engineering, Horticulture & Bedding, Coir Twine & Yarn, Growing Media', metadata={'symbol': 'HEXP', 'name': 'HAYLEYS FIBRE PLC'}), -0.03889149311133844), (Document(page_content='MYLAND DEVELOPMENTS PLC Real Estate and Construction.', metadata={'symbol': 'MDL', 'name': 'MYLAND DEVELOPMENTS PLC'}), -0.06313638342353323), (Document(page_content='C I C HOLDINGS PLC merchandising and manufacturing. The Company’s segments include Crop Solutions, Agri-Produce, Livestock Solutions, Industrial Solutions and Health & Personal care.', metadata={'symbol': 'CIC', 'name':

10
EXPO EXPOLANKA HOLDINGS PLC
returned :  15
15
LIOC LANKA IOC PLC
returned :  15
30
ACL ACL CABLES PLC
returned :  15
45
LOFC LOLC FINANCE PLC
returned :  15
60
RCL ROYAL CERAMICS LANKA PLC
returned :  15
75
LOLC L O L C HOLDINGS PLC
returned :  15
90
HAYL HAYLEYS PLC
returned :  15
105
CIC C I C HOLDINGS PLC
returned :  15
120
VONE VALLIBEL ONE PLC
returned :  15
135
SAMP SAMPATH BANK PLC
returned :  15
150
170


In [ ]:
with gr.Blocks() as demo:
    input = gr.Dropdown(
        ["finance", "transport", "technology", "agriculture"],
        multiselect=True,
        label="Domains",
        allow_custom_value=True,
        scale=5
    )

    button = gr.Button("Generate Recommendations")

    with gr.Column() as output_col:
        with gr.Accordion(open = False) as acc1:
            with gr.Row():
                txt1 = gr.TextArea()
                plt1 = gr.Plot()
        with gr.Accordion(open = False) as acc2:
            with gr.Row():
                txt2 = gr.TextArea()
                plt2 = gr.Plot()
        with gr.Accordion(open = False) as acc3:
            with gr.Row():
                txt3 = gr.TextArea()
                plt3 = gr.Plot()
        with gr.Accordion(open = False) as acc4:
            with gr.Row():
                txt4 = gr.TextArea()
                plt4 = gr.Plot()
        with gr.Accordion(open = False) as acc5:
            with gr.Row():
                txt5 = gr.TextArea()
                plt5 = gr.Plot()
        with gr.Accordion(open = False) as acc6:
            with gr.Row():
                txt6 = gr.TextArea()
                plt6 = gr.Plot()
        with gr.Accordion(open = False) as acc7:
            with gr.Row():
                txt7 = gr.TextArea()
                plt7 = gr.Plot()
        with gr.Accordion(open = False) as acc8:
            with gr.Row():
                txt8 = gr.TextArea()
                plt8 = gr.Plot()
        with gr.Accordion(open = False) as acc9:
            with gr.Row():
                txt9 = gr.TextArea()
                plt9 = gr.Plot()
        with gr.Accordion(open = False) as acc10:
            with gr.Row():
                txt10 = gr.TextArea()
                plt10 = gr.Plot()


    outputs = [acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,
            txt1,txt2,txt3,txt4,txt5,txt6,txt7,txt8,txt9,txt10,
            plt1,plt2,plt3,plt4,plt5,plt6,plt7,plt8,plt9,plt10]

    button.click(fn = get_recommendations_gradio, inputs = input, outputs = outputs)

demo.launch()

In [ ]:
with gr.Blocks() as demo:
    inputs = gr.Dropdown(
        ["finance", "transport", "technology", "agriculture"],
        multiselect=True,
        label="Domains",
        allow_custom_value=True,
        scale=5
    )
    
    button = gr.Button("Generate Recommendations")

    outputs = [gr.Plot(), gr.Plot(),gr.Plot(),
               gr.Plot(),gr.Plot(),gr.Plot(),
               gr.Plot(),gr.Plot(),gr.Plot(),gr.Plot()]

    button.click(get_recommendations_gradio, inputs=inputs, outputs=outputs)
    

    demo.launch(share = True)

# Sandbox